### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [34]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## Local Government Area Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average maths score 

* Calculate the average reading score

* Calculate the percentage of students with a passing maths score (50 or greater)

* Calculate the percentage of students with a passing reading score (50 or greater)

* Calculate the percentage of students who passed maths **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [35]:
# Define the Calculations
total_schools = school_data["School ID"].nunique()
total_students = student_data["Student ID"].nunique()
total_budget = (school_data["budget"].sum())
avg_maths = (student_data["maths_score"].mean())
avg_reading = (student_data["reading_score"].mean())
maths_pass = (student_data["Student ID"].loc[student_data["maths_score"] >=50].count()/student_data["Student ID"].nunique()*100)
reading_pass = (student_data["Student ID"].loc[student_data["reading_score"] >=50].count()/student_data["Student ID"].nunique()*100)
maths_reading_pass = (student_data["Student ID"].loc[(student_data["maths_score"] >=50) & (student_data["reading_score"] >=50)].count()/student_data["Student ID"].nunique()*100)

# Setup the dataframe
area_summary  = [{"Total Schools":total_schools,\
                                 "Total Students":total_students,\
                                 "Total Budget":total_budget,\
                                 "Average Maths":avg_maths,\
                                 "Average Reading":avg_reading,\
                                 "Maths Pass %":maths_pass,\
                                 "Reading Pass %":reading_pass,\
                                 "Both Pass %":maths_reading_pass}]
area_summary  = pd.DataFrame(area_summary )

# Format the values
area_summary ["Total Students"] = area_summary ["Total Students"].map("{:,}".format)
area_summary ["Total Budget"] = area_summary ["Total Budget"].map("${:,.2f}".format)
area_summary ["Average Maths"] = area_summary ["Average Maths"].map("{:.2f}".format)
area_summary ["Average Reading"] = area_summary ["Average Reading"].map("{:.2f}".format)
area_summary ["Maths Pass %"] = area_summary ["Maths Pass %"].map("{:,.2f}%".format)
area_summary ["Reading Pass %"] = area_summary ["Reading Pass %"].map("{:,.2f}%".format)
area_summary ["Both Pass %"] = area_summary ["Both Pass %"].map("{:,.2f}%".format)


area_summary 

,Total Schools,Total Students,Total Budget,Average Maths,Average Reading,Maths Pass %,Reading Pass %,Both Pass %
0,15,"39,170","$24,649,428.00",70.34,69.98,86.08%,84.43%,72.81%


## School Summary

* Create an overview table that summarises key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Maths Score
  * Average Reading Score
  * % Passing Maths
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed maths **and** reading.)
  
* Create a dataframe to hold the above results

In [36]:
#Set up the conditions for the new columns
school_data_complete["% Passing Reading"] = np.where(school_data_complete["reading_score"]>=50, 1, 0)
school_data_complete["% Passing Maths"] = np.where(school_data_complete["maths_score"]>=50, 1, 0)
school_data_complete["% Overall Passing"] = np.where((school_data_complete["maths_score"]>=50) & (school_data_complete["reading_score"]>=50), 1, 0)


#Aggregate the dataframe
school_data_summary = school_data_complete.groupby(["school_name","type","size","budget"],as_index=False)\
    .agg({'reading_score':'mean', 'maths_score':'mean', "% Passing Reading":'sum', "% Passing Maths":'sum', "% Overall Passing":'sum'})\
    .rename(columns={'reading_score':'Average Reading Score', 'maths_score':'Average Maths Score', 'school_name':'School Name',\
                     'type':'School Type', 'size': 'Total Students', 'budget':'Total School Budget'})

# Add new columns
school_data_summary["% Passing Maths"] = school_data_summary["% Passing Maths"]/school_data_summary["Total Students"]*100
school_data_summary["% Passing Reading"] = school_data_summary["% Passing Reading"]/school_data_summary["Total Students"]*100
school_data_summary["% Overall Passing"] = school_data_summary["% Overall Passing"]/school_data_summary["Total Students"]*100
school_data_summary["Budget per student"] = school_data_summary["Total School Budget"]/school_data_summary["Total Students"]

#Format columns
school_data_summary["Total School Budget"] = school_data_summary["Total School Budget"].map("${:,.2f}".format)
school_data_summary["Per Student Budget"] = school_data_summary["Budget per student"].map("${:,.2f}".format)

school_data_summary.head()

,School Name,School Type,Total Students,Total School Budget,Average Reading Score,Average Maths Score,% Passing Reading,% Passing Maths,% Overall Passing,Budget per student,Per Student Budget
0,Bailey High School,Government,4976,"$3,124,928.00",71.008842,72.352894,87.379421,91.639871,80.084405,628.0,$628.00
1,Cabrera High School,Independent,1858,"$1,081,356.00",71.359526,71.657158,89.074273,90.850377,80.785791,582.0,$582.00
2,Figueroa High School,Government,2949,"$1,884,411.00",69.077993,68.698542,82.807731,81.654798,67.650051,639.0,$639.00
3,Ford High School,Government,2739,"$1,763,916.00",69.572472,69.091274,82.219788,82.438846,67.469880,644.0,$644.00
4,Griffin High School,Independent,1468,"$917,500.00",71.245232,71.788147,88.487738,91.212534,81.335150,625.0,$625.00


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [37]:
top_schools = school_data_summary.sort_values("% Overall Passing",ascending=False)
top_schools .head()

,School Name,School Type,Total Students,Total School Budget,Average Reading Score,Average Maths Score,% Passing Reading,% Passing Maths,% Overall Passing,Budget per student,Per Student Budget
4,Griffin High School,Independent,1468,"$917,500.00",71.245232,71.788147,88.487738,91.212534,81.335150,625.0,$625.00
1,Cabrera High School,Independent,1858,"$1,081,356.00",71.359526,71.657158,89.074273,90.850377,80.785791,582.0,$582.00
0,Bailey High School,Government,4976,"$3,124,928.00",71.008842,72.352894,87.379421,91.639871,80.084405,628.0,$628.00
14,Wright High School,Independent,1800,"$1,049,400.00",70.969444,72.047222,86.666667,91.777778,79.722222,583.0,$583.00
10,Rodriguez High School,Government,3999,"$2,547,363.00",70.935984,72.047762,87.396849,90.797699,79.419855,637.0,$637.00


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [38]:
bottom_schools = school_data_summary.sort_values("% Overall Passing",ascending=True)
bottom_schools .head()

,School Name,School Type,Total Students,Total School Budget,Average Reading Score,Average Maths Score,% Passing Reading,% Passing Maths,% Overall Passing,Budget per student,Per Student Budget
5,Hernandez High School,Government,4635,"$3,022,020.00",69.186408,68.874865,81.877023,80.949299,66.364617,652.0,$652.00
7,Huang High School,Government,2917,"$1,910,635.00",68.910525,68.935207,81.453548,81.693521,66.712376,655.0,$655.00
8,Johnson High School,Government,4761,"$3,094,650.00",69.039277,68.843100,81.978576,82.062592,67.191766,650.0,$650.00
13,Wilson High School,Independent,2283,"$1,319,574.00",68.876916,69.170828,81.296540,82.785808,67.455103,578.0,$578.00
3,Ford High School,Government,2739,"$1,763,916.00",69.572472,69.091274,82.219788,82.438846,67.469880,644.0,$644.00


## Maths Scores by Year

* Create a table that lists the average maths score for students of each year level (9, 10, 11, 12) at each school.

  * Create a pandas series for each year. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [39]:
# Define new Dataframe
maths_scores_by_year  = school_data_complete [["school_name","year","maths_score"]]

# Group the new Dataframe and then rename the columns
maths_scores_by_year  _data_year=maths_scores_by_year .groupby(["school_name","year"],as_index=False)\
    .agg({'maths_score':'mean'})\
    .rename(columns={'maths_score':'Mean Maths Score'})

#Rearrange the format of the Dataframe
maths_scores_by_year  = maths_scores_by_year .pivot_table('Mean Maths Score', ['school_name'], 'year')
maths_scores_by_year .head(15)

SyntaxError: invalid syntax (2973807116.py, line 5)

## Reading Score by Year

* Perform the same operations as above for reading scores

In [40]:
# Define new Dataframe
reading_scores_by_year  = school_data_complete [["school_name","year","reading_score"]]

# Group the new Dataframe and then rename the columns
reading_scores_by_year =reading_scores_by_year .groupby(["school_name","year"],as_index=False)\
    .agg({'reading_score':'mean'})\
    .rename(columns={'reading_score':'Mean Reading Score'})

#Rearrange the format of the Dataframe
reading_scores_by_year  = reading_scores_by_year .pivot_table('Mean Reading Score', ['school_name'], 'year')
reading_scores_by_year .head(15)

year,9,10,11,12
school_name,,,,
Bailey High School,70.901920,70.848265,70.317346,72.195525
Cabrera High School,71.172348,71.328326,71.201245,71.856021
Figueroa High School,70.261682,67.677588,69.152327,69.082126
Ford High School,69.615846,68.988701,70.735964,68.849722
Griffin High School,72.026895,70.746305,72.385042,69.434932
Hernandez High School,68.477569,70.621842,68.418199,69.244136
Holden High School,71.598425,71.096491,73.310680,70.481928
Huang High School,68.670616,69.516297,68.740638,68.671795
Johnson High School,68.719286,69.295029,69.969115,67.992521


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Maths Score
  * Average Reading Score
  * % Passing Maths
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [41]:
# Define Bins and Group names
bins = [1, 585, 630, 645, 680]
group_names = ["<$585", "$585-630", "$630-645", "$645-680"]

# Create new Dataframe
spending_summary  = school_data_summary
spending_summary ["Spending Ranges (Per Student)"] = pd.cut(spending_summary ["Budget per student"], bins, labels=group_names, include_lowest=True)

spending_summary  = spending_summary .drop(['School Name', 'School Type','Total Students', 'Total School Budget','Budget per student'], axis=1)
spending_summary .insert(0, "Spending Ranges (Per Student)", spending_summary .pop("Spending Ranges (Per Student)"))

# Group the new Dataframe
spending_summary =spending_summary .groupby(["Spending Ranges (Per Student)"],as_index=False)\
      .agg({'Average Reading Score':'mean','Average Maths Score':'mean','% Passing Reading':'mean','% Passing Maths':'mean','% Overall Passing':'mean'})
spending_summary.head()

,Spending Ranges (Per Student),Average Reading Score,Average Maths Score,% Passing Reading,% Passing Maths,% Overall Passing
0,<$585,70.716577,71.364587,86.390517,88.835926,76.721458
1,$585-630,71.031297,72.065868,87.292423,91.518824,79.876293
2,$630-645,69.838814,69.854807,83.763585,84.686139,71.004977
3,$645-680,69.045403,68.884391,81.769716,81.568470,66.756253


## Scores by School Size

* Perform the same operations as above, based on school size.

In [42]:
# Define Bins and Group names
size_bins = [0, 1000, 2000, 5000]
label = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

# Create new Dataframe
size_summary  = school_data_summary
size_summary ["Size Category"] = pd.cut(size_summary ["Total Students"],size_bins, labels=label, include_lowest = False)
size_summary =size_summary .drop(['School Name', 'School Type','Total Students', 'Total School Budget','Budget per student'], axis=1)
size_summary .insert(0, 'Size Category', size_summary .pop('Size Category'))

# Group the new Dataframe
size_summary =size_summary .groupby(["Size Category"],as_index=False)\
      .agg({'Average Reading Score':'mean','Average Maths Score':'mean','% Passing Reading':'mean','% Passing Maths':'mean','% Overall Passing':'mean'})
size_summary .head()

,Size Category,Average Reading Score,Average Maths Score,% Passing Reading,% Passing Maths,% Overall Passing
0,Small (<1000),71.636864,72.335748,87.557513,90.806867,79.066348
1,Medium (1000-2000),70.720164,71.421650,86.714149,89.846560,78.039785
2,Large (2000-5000),69.576052,69.751809,83.301185,84.252804,70.293507


## Scores by School Type

* Perform the same operations as above, based on school type

In [43]:
# Define new Dataframe
type_summary  = school_data_summary
type_summary =type_summary .drop(['School Name', 'Total School Budget','Per Student Budget','Total Students','% Overall Passing'], axis=1)

# Group the new Dataframe
type_summary =type_summary .groupby(["School Type"],as_index=False)\
      .agg({'Average Reading Score':'mean','Average Maths Score':'mean','% Passing Reading':'mean','% Passing Maths':'mean'})
type_summary .head(50)

,School Type,Average Reading Score,Average Maths Score,% Passing Reading,% Passing Maths
0,Government,69.675929,69.834806,83.587562,84.462375
1,Independent,70.718933,71.368822,86.247789,89.204043
